In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set()
sns.set(style='darkgrid')
from matplotlib import pyplot as plt
import plotly.graph_objects as go

# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz
# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read the Indian Food 101 Dataset
df_101 = pd.read_csv('/kaggle/input/indian-food-101/indian_food.csv')
# Display Top 5 rows
df_101.head()

In [ ]:
# Read 6000 Indian Food Recipes Dataset
df_6000 = pd.read_csv('/kaggle/input/6000-indian-food-recipes-dataset/IndianFoodDatasetCSV.csv')
# Display top 5 rows
df_6000.head()

In [ ]:
# Total number of recipes by state from Indian Food 101 dataset
df_101.state.value_counts()

In [ ]:
# Extract only recipes belong to Tamil Nadu
df_tn = df_101[df_101['state'] == 'Tamil Nadu']
df_tn

# Let's choose a dish from 101 dataset and extract different varieties of the same from 6000 dataset

## We all love varieties in our faood :-) 

In [ ]:
# Use process to extract top 20 matches for the recipe pongal
possibilities = process.extract("pongal", df_6000['TranslatedRecipeName'], limit=20, scorer=fuzz.partial_ratio)

In [ ]:
# Get matches with partial ratio as 100
# Partial rtaio compare subsections of the string
extracted_matches = [possible for possible in possibilities if possible[1] == 100]
extracted_matches

In [ ]:
# Extract only the recipe name from the result
get_recipe_name = [extracted_matches[i][0] for i in range(len(extracted_matches))]

In [ ]:
# Get recipes of the extracted matches
df_updated = df_6000[df_6000['TranslatedRecipeName'].isin(get_recipe_name)]

# Select meal based on prep time

In [ ]:
a=sns.barplot(y='TranslatedRecipeName',x='TotalTimeInMins',data=df_updated)

In [ ]:
print("What's the time you can wait before eating?")
print("1.Fast and Delicious\n\n2.I live for food\n\nEnter the number")
x=int(input())
if x==1:
    print(df_updated['TranslatedRecipeName'][df_updated[df_updated['TotalTimeInMins'] <= 40].index.tolist()].tolist())
else:
    print(df_updated['TranslatedRecipeName'][df_updated[df_updated['TotalTimeInMins'] > 40].index.tolist()].tolist())

In [ ]:
# Explore commonly used ingredient in tamil nadu food
df_tn['ingredients'] = df_tn['ingredients'].str.split(',')

In [ ]:
df_tn

In [ ]:
df_101['state'].unique()

In [ ]:
# Get unique ingredients with count
plt.figure(figsize=(15,16))
def get_unique_ingredients(state):
    df_unique = df_101[df_101['state'] == state]
    df_unique['ingredients'] = df_unique['ingredients'].str.split(',')
    print(df_unique['ingredients'].explode().unique())
    fig = go.Figure(data=[go.Pie(labels=df_unique['ingredients'].explode().unique(), values=df_unique['ingredients'].explode().value_counts())])
    fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))
    fig.show()
    
get_unique_ingredients('Tamil Nadu')